In [13]:
import os
import itertools
#from rmgpy.molecule import Molecule

directory = '../'

species = {
  
  #'C2H6' : 'CC',
  #'CH4': 'C',
  'CH2F2': 'FCF',
  'CH3F': 'CF',
  'C2H5F': 'CCF',
  'CH2FCH2F': 'FCCF',
  'CH2FCHF2' : 'FCC(F)F',
  'CH3CF3': 'CC(F)(F)F',
  'CH3CHF2': 'CC(F)F',
}


combinations = []



#for loop where I can couple halocarbons
temp_species_list = list(species.keys())
for idx, x in enumerate(temp_species_list): 
    other_species = temp_species_list[:idx] + temp_species_list[idx+1:]   
    for idi, i in enumerate(other_species):
        combinations.append([x,i])
#this print statement better be 42 or im gonna be really upset 
print(len(combinations))
print(combinations)

for x in combinations: 
    label = []
    smiles = []
    label.append(x[0])
    label.append(x[1]) 
    smiles.append(species[x[0]]) 
    smiles.append(species[x[1]])
    print(label, smiles)

42
[['CH2F2', 'CH3F'], ['CH2F2', 'C2H5F'], ['CH2F2', 'CH2FCH2F'], ['CH2F2', 'CH2FCHF2'], ['CH2F2', 'CH3CF3'], ['CH2F2', 'CH3CHF2'], ['CH3F', 'CH2F2'], ['CH3F', 'C2H5F'], ['CH3F', 'CH2FCH2F'], ['CH3F', 'CH2FCHF2'], ['CH3F', 'CH3CF3'], ['CH3F', 'CH3CHF2'], ['C2H5F', 'CH2F2'], ['C2H5F', 'CH3F'], ['C2H5F', 'CH2FCH2F'], ['C2H5F', 'CH2FCHF2'], ['C2H5F', 'CH3CF3'], ['C2H5F', 'CH3CHF2'], ['CH2FCH2F', 'CH2F2'], ['CH2FCH2F', 'CH3F'], ['CH2FCH2F', 'C2H5F'], ['CH2FCH2F', 'CH2FCHF2'], ['CH2FCH2F', 'CH3CF3'], ['CH2FCH2F', 'CH3CHF2'], ['CH2FCHF2', 'CH2F2'], ['CH2FCHF2', 'CH3F'], ['CH2FCHF2', 'C2H5F'], ['CH2FCHF2', 'CH2FCH2F'], ['CH2FCHF2', 'CH3CF3'], ['CH2FCHF2', 'CH3CHF2'], ['CH3CF3', 'CH2F2'], ['CH3CF3', 'CH3F'], ['CH3CF3', 'C2H5F'], ['CH3CF3', 'CH2FCH2F'], ['CH3CF3', 'CH2FCHF2'], ['CH3CF3', 'CH3CHF2'], ['CH3CHF2', 'CH2F2'], ['CH3CHF2', 'CH3F'], ['CH3CHF2', 'C2H5F'], ['CH3CHF2', 'CH2FCH2F'], ['CH3CHF2', 'CH2FCHF2'], ['CH3CHF2', 'CH3CF3']]
['CH2F2', 'CH3F'] ['FCF', 'CF']
['CH2F2', 'C2H5F'] ['FCF', '

In [14]:
input_file= '''
thermolibs = [
'primaryThermoLibrary',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'Fluorine',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

thermolibs_Creg = [
'primaryThermoLibrary',
'FFCM1(-)',
'DFT_QCI_thermo',
]


'''


def add_database(labels): 

    block = '''
database(
thermoLibraries = thermolibs,
reactionLibraries = ['FFCM1(-)','halogens_pdep'],
seedMechanisms = ['FFCM1(-)'],
kineticsDepositories = ['training'],
kineticsFamilies = ['default','halogens','Disproportionation-Y'],
frequenciesLibraries = ['halogens_G4'],
kineticsEstimator = 'rate rules',
)
'''
    return block

def make_species_block(labels):
    """
    Makes the species block for the list of labels
    """
    block = ""
    
    for label in labels:
        smiles = species[label]
        block += \
        f'''
species(
    label = '{label}',
    reactive = {reactive},
    structure = SMILES('{smiles}')
)'''

    block += '''
species(
    label = 'O2',
    reactive = True,
    structure = SMILES('[O][O]')
)

species(
    label = 'CH4',
    reactive = True,
    structure = SMILES('C')
)

species(
    label = 'N2',
    reactive = False,
    structure = SMILES('N#N')
)

   '''
    return block

def make_reactor(labels, ratios=None):
    """
    Creates a single reactor block for the given ratios.
    if you supply ratios, its for the moles of halocarbons.
    To these are added 1 mol O2 and a balance of N2.
    
    #todo: consider stoiciometry, and maybe do range reactor on composition
    """
    if ratios:
        assert len(labels) == len(ratios), "should have same number of ratios as labels"
    else:
        ratios = [1./len(labels) for i in labels] # equal if undefined

    reactor = \
    '''
simpleReactor(
        temperature=[(1000,'K'),(2000,'K')],
        pressure= [(1.0,'bar'),(10.0,'bar')],
        nSims=10,
        initialMoleFractions={'''
    for label, ratio in zip(labels, ratios):
        reactor += f'"{label}": {ratio},\n'
    reactor += '''
        "O2": 1,
        "N2": 3.76,
        },
        # terminationConversion={
        # 'halogen': 0.999,
        # },
        #terminationRateRatio=1e-4,
        #terminationTime=(10,'s'),
        terminationTime=(1,'s'),
        #sensitivity=['OH'],
        #sensitivityThreshold=0.001,
        )
        '''
    return reactor

def make_generated_species_constraints():
    block = \
    '''
    
simulator(
    atol = 1e-16,
    rtol = 1e-08,
    sens_atol = 1e-06,
    sens_rtol = 0.0001,
)


generatedSpeciesConstraints(
    allowed=['input species','seed mechanisms','reaction libraries'],
    maximumCarbonAtoms=8,
    maximumOxygenAtoms=6,
    #maximumHeavyAtoms=24,
    maximumRadicalElectrons=2,
    maximumSingletCarbenes=1,
    maximumCarbeneRadicals=0,
    allowSingletO2 = True,
)

options(
    units = "si",
    generateSeedEachIteration = True,
    generateOutputHTML = True,
    generatePlots = True,
    saveSimulationProfiles = True,
    saveEdgeSpecies = False,
    keepIrreversible = True,
    verboseComments = False,
)
    
    '''
    return block 


def make_bash_script(labels):
    job_name = '/'.join(labels)
    bash_script = """#!/bin/sh

#SBATCH --nodes=1
#SBATCH --time=12-00:00:00
#SBATCH --job-name=job_name
#SBATCH --error=rmg.slurm.log
#SBATCH --cpus-per-task=5
#SBATCH --mem-per-cpu=8Gb
#SBATCH --ntasks=1 
#SBATCH --array=1
#SBATCH --partition=west

source activate rmg_env
python-jl /home/khalil.nor/Code/RMG-Py/rmg.py input.py
""".replace('job_name',job_name)
    return bash_script

model_loose = \
"""
model(
    toleranceMoveToCore = 0.5,
    toleranceInterruptSimulation = 0.5,
    toleranceKeepInEdge = 0.01,
    maximumEdgeSpecies = 5e5,
    filterReactions = True,
    filterThreshold = 5e8,
    minCoreSizeForPrune = 100,
    minSpeciesExistIterationsForPrune = 2,
)
"""

model_medium = \
"""
model(
    toleranceMoveToCore = 0.2,
    toleranceInterruptSimulation = 0.2,
    maximumEdgeSpecies = 5e5,
    filterReactions = True,
    filterThreshold = 5e8,
    minCoreSizeForPrune = 50,
    minSpeciesExistIterationsForPrune = 4,
)
"""

model_tight = \
"""
model(
    toleranceMoveToCore = 0.1,
    toleranceInterruptSimulation = 0.1,
    maximumEdgeSpecies = 3e5,
    filterReactions = True,
    filterThreshold = 5e8,
    minCoreSizeForPrune = 50,
    minSpeciesExistIterationsForPrune = 4,
)
"""

pdep = \
"""
pressureDependence(
    method='modified strong collision',
    maximumGrainSize=(0.5,'kcal/mol'),
    minimumNumberOfGrains=250,
    temperatures=(300,2500,'K',8),
    pressures=(0.01,100,'bar',5),
    interpolation=('Chebyshev', 6, 4),
    maximumAtoms=16,
)
"""










['CH2F2', 'CH3F'] ['FCF', 'CF']
CF
['CH2F2', 'C2H5F'] ['FCF', 'CCF']
CCF
['CH2F2', 'CH2FCH2F'] ['FCF', 'FCCF']
FCCF
['CH2F2', 'CH2FCHF2'] ['FCF', 'FCC(F)F']
FCC(F)F
['CH2F2', 'CH3CF3'] ['FCF', 'CC(F)(F)F']
CC(F)(F)F
['CH2F2', 'CH3CHF2'] ['FCF', 'CC(F)F']
CC(F)F
['CH3F', 'CH2F2'] ['CF', 'FCF']
FCF
['CH3F', 'C2H5F'] ['CF', 'CCF']
CCF
['CH3F', 'CH2FCH2F'] ['CF', 'FCCF']
FCCF
['CH3F', 'CH2FCHF2'] ['CF', 'FCC(F)F']
FCC(F)F
['CH3F', 'CH3CF3'] ['CF', 'CC(F)(F)F']
CC(F)(F)F
['CH3F', 'CH3CHF2'] ['CF', 'CC(F)F']
CC(F)F
['C2H5F', 'CH2F2'] ['CCF', 'FCF']
FCF
['C2H5F', 'CH3F'] ['CCF', 'CF']
CF
['C2H5F', 'CH2FCH2F'] ['CCF', 'FCCF']
FCCF
['C2H5F', 'CH2FCHF2'] ['CCF', 'FCC(F)F']
FCC(F)F
['C2H5F', 'CH3CF3'] ['CCF', 'CC(F)(F)F']
CC(F)(F)F
['C2H5F', 'CH3CHF2'] ['CCF', 'CC(F)F']
CC(F)F
['CH2FCH2F', 'CH2F2'] ['FCCF', 'FCF']
FCF
['CH2FCH2F', 'CH3F'] ['FCCF', 'CF']
CF
['CH2FCH2F', 'C2H5F'] ['FCCF', 'CCF']
CCF
['CH2FCH2F', 'CH2FCHF2'] ['FCCF', 'FCC(F)F']
FCC(F)F
['CH2FCH2F', 'CH3CF3'] ['FCCF', 'CC(F)(F)F']
CC

In [ ]:


def make_blend{labels}:
    """Provided with an iterator of labels, makes a blend folder and files.
    
    example: make_blend(['CH3CHF2', 'C2H5F'])
    """

    d = os.path.join(directory, '_'.join(labels)))
    os.makedirs(d, exist_ok=True)
    
    input0 = input_file
    input0 += add_database(labels)
    species_block = ""
    species_block += make_species_block(labels)
    input0 += species_block
    input0 += make_generated_species_constraints()
    input0 += make_reactor(labels)
    input0 += model_tight
    input0 += pdep

    with open(os.path.join(d,'input.py'),'w') as f:
    for l in input0:
        f.write(l)
            
    bash_script = make_bash_script(labels)

    with open(os.path.join(d,'run.sh'),'w') as f:
        for l in bash_script:
            f.write(l)
    

In [2]:
if __name__ == "__main__":

    for combination in itertools.combinations(species, 2):
    print(combination)
    make_blend(combination)
    

ValueError: factorial() not defined for negative values